# Overview

* This notebook covers how to utilize Sagemaker NEO and Sagemaker Elastic Inference (EI)
* In this example, we build a ResNet transfer learning model to predict hot dog/not hot dog [a la Silicon Valley](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=2ahUKEwieq5LR5_jkAhXFY98KHdvcBXEQwqsBMAB6BAgJEAQ&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DACmydtFDTGs&usg=AOvVaw1OQiCPPVe3B2B6ndhvDGnq)
* Note to run through this notebook Sagemaker P2 instances and a notebook with at least 10 gb of disk space is needed
* We utilize the Food101 dataset to create the hot dog/not hot dog dataset
* By the end of the notebook we show how inference speed are measured against cost for a ResNet Model


# Data Prep

* Download and unzip the Food-101 dataset in a terminal using the code below 

 `cd SageMaker/MxNet-Sagemaker-Deployments`

 `wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz`
 
 `tar -zxvf food-101.tar.gz`

In [1]:
# load necessary packages
import pandas as pd 
import json
from glob import glob
import shutil
import os
from mxnet import gluon

In [42]:
# Change directory to food-101 meta
os.chdir('food-101')

In [20]:
train_json = json.load(open('meta/train.json'))
test_json = json.load(open('meta/test.json'))

In [21]:
# make directories for the hot dog/not hot dog dataset
try:
    os.mkdir('../hotdog_not_hotdog/')

    os.mkdir('../hotdog_not_hotdog/train/')
    os.mkdir('../hotdog_not_hotdog/test/')

    os.mkdir('../hotdog_not_hotdog/train/hot_dog/')
    os.mkdir('../hotdog_not_hotdog/test/hot_dog/')

    os.mkdir('../hotdog_not_hotdog/train/not_hotdog/')
    os.mkdir('../hotdog_not_hotdog/test/not_hotdog/')
except FileExistsError:
    pass

In [38]:
import random
import copy

def move_and_rename(json, dest, n_images):
    '''
    This function takes a json of file names, copies and renames these files into new directories
    All images are copied for hot dog files, the function randomly copies other images for number of n_images
    json : dict, dict of filenames
    dest, string, local folder where to deposit files
    n_images, int, number of images to randomly sample for not hot dog images
    '''
    json_copy = copy.deepcopy(json)
    hotdog_images = json_copy['hot_dog']
    for i in hotdog_images:
        shutil.copyfile('images/{}.jpg'.format(i), '../hotdog_not_hotdog/{}/{}.jpg'.format(dest,i))
    json_copy.pop('hot_dog')
    other_foods = list(json_copy.keys())
    cnt = 0
    for i in range(n_images):
        random_indexer = random.randint(0, len(other_foods)-1)
        print(other_foods[random_indexer])
        other_class_imgs = json_copy[other_foods[random_indexer]]
        img_indexer = random.randint(0, len(other_class_imgs)-1)
        selected_image = other_class_imgs[img_indexer]
        destination_name = 'not_hotdog/{}'.format(cnt)
        shutil.copyfile('images/{}.jpg'.format(selected_image), '../hotdog_not_hotdog/{}/{}.jpg'.format(dest,destination_name))
        other_class_imgs.pop(img_indexer)
        # delete used image from list of possibilities
        json_copy[other_foods[random_indexer]] = other_class_imgs
        cnt += 1

In [44]:
# create dataset folders
move_and_rename(train_json, 'train', 750)
move_and_rename(test_json, 'test', 250)

ceviche
prime_rib
peking_duck
gnocchi
pad_thai
eggs_benedict
chicken_curry
poutine
foie_gras
fried_rice
peking_duck
garlic_bread
cheesecake
dumplings
club_sandwich
french_toast
deviled_eggs
pho
baby_back_ribs
beef_tartare
crab_cakes
pork_chop
falafel
ceviche
caprese_salad
sashimi
foie_gras
risotto
waffles
donuts
prime_rib
churros
takoyaki
bibimbap
club_sandwich
tiramisu
french_fries
huevos_rancheros
edamame
waffles
grilled_salmon
seaweed_salad
takoyaki
cannoli
croque_madame
beignets
chocolate_cake
greek_salad
beef_tartare
pulled_pork_sandwich
chicken_curry
omelette
churros
huevos_rancheros
deviled_eggs
baby_back_ribs
beef_tartare
gyoza
beef_tartare
dumplings
frozen_yogurt
eggs_benedict
ice_cream
foie_gras
omelette
sushi
red_velvet_cake
nachos
french_toast
foie_gras
cup_cakes
miso_soup
omelette
panna_cotta
bibimbap
prime_rib
ramen
chocolate_mousse
poutine
chicken_wings
cup_cakes
chicken_wings
guacamole
hot_and_sour_soup
red_velvet_cake
frozen_yogurt
panna_cotta
beet_salad
bruschetta
ris

In [45]:
#validate the number of images in the folders
len(glob('../hotdog_not_hotdog/test/hot_dog/*'))

250

In [46]:
len(glob('../hotdog_not_hotdog/test/not_hotdog/*'))

250

# Model Training

## Setup

Create Sagemaker session and role

In [28]:
import sagemaker
from sagemaker.mxnet import MXNet

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

## Upload Data

* Sagemaker expects the data to be in an s3 path

In [29]:
os.chdir('/home/ec2-user/SageMaker/MxNet-Sagemaker-Deployments/')

In [30]:
inputs = sagemaker_session.upload_data(path='hotdog_not_hotdog', key_prefix='data/DEMO-hotdog_not_hotdog')
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-532389884884/data/DEMO-hotdog_not_hotdog


## Notes on the MxNet Script

* The 'hotdog_v3.py' file has functions for training and deploying the model
* Note that the model has the following hyperparameters for training
    * batch_size, int, number for batch size
    * epochs, int, number of epochs to run training
    * learning rate, float, the learning rate for the model
    * momentum, float, momentum for the SGD algorithm
    * wd, float, weight decay parameter for model params
    * resnet_size, str, size of resnet to use one of 18, 34, 50, 101, 151
    

* As a opposed to a standard MxNet script to use Sagemaker NEO special functions need to be added
    * These are seen at the bottom of the script (neo_postprocess and neo_preprocess)

In [31]:
!pygmentize 'hotdog_v3.py'

from __future__ import print_function

import json
import logging
import os
import shutil
import time
import warnings
import io
import argparse

import mxnet as mx
from mxnet import autograd as ag
from mxnet import gluon
from mxnet.gluon.model_zoo import vision as models
import numpy as np

from mxnet.gluon.data.vision.datasets import ImageFolderDataset
from mxnet.gluon.data.vision import transforms


# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #

def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--batch_size', type=int, default=128)
    parser.add_argument('--epochs', type=int, default=10)
    parser.add_argument('--learning_rate', type=float, default=0.1)
    parser.add_argument('--momentum', type=float, default=0.9)
    parser.add_argument('--wd', type=float, default=0.0001)
    parser.add_argument('--re

## Training Job

* Instantiate the Sagemaker MxNet estimator with the role, instance type, number of instances and hyperparameters

In [32]:
m = MXNet('hotdog_v3.py',
          role=role, 
          framework_version='1.4.1',
          train_instance_count=1,
          train_instance_type='ml.p2.xlarge',
          py_version='py3',
          hyperparameters={'batch_size': 32,
                           'epochs': 5,
                           'learning_rate': 0.01,
                           'momentum': 0.9,
                           'resnet_size':'101'})

* Fit the model against the s3 path specified earlier

In [33]:
m.fit(inputs)

2019-10-07 18:42:43 Starting - Starting the training job...
2019-10-07 18:42:47 Starting - Launching requested ML instances.........
2019-10-07 18:44:18 Starting - Preparing the instances for training......
2019-10-07 18:45:39 Downloading - Downloading input data......
2019-10-07 18:46:43 Training - Training image download completed. Training in progress..2019-10-07 18:46:44,080 sagemaker-containers INFO     Imported framework sagemaker_mxnet_container.training
2019-10-07 18:46:44,107 sagemaker_mxnet_container.training INFO     MXNet training environment: {'SM_HOSTS': '["algo-1"]', 'SM_NETWORK_INTERFACE_NAME': 'eth0', 'SM_HPS': '{"batch_size":32,"epochs":5,"learning_rate":0.01,"momentum":0.9,"resnet_size":"101"}', 'SM_USER_ENTRY_POINT': 'hotdog_v3.py', 'SM_FRAMEWORK_PARAMS': '{}', 'SM_RESOURCE_CONFIG': '{"current_host":"algo-1","hosts":["algo-1"],"network_interface_name":"eth0"}', 'SM_INPUT_DATA_CONFIG': '{"training":{"RecordWrapperType":"None","S3DistributionType":"FullyReplicated","T

Transformed Training and Test Files
Initialized Batching Operation
/usr/local/lib/python3.6/site-packages/mxnet/gluon/parameter.py:813: UserWarning: Parameter 'resnetv20_batchnorm0_gamma' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/usr/local/lib/python3.6/site-packages/mxnet/gluon/parameter.py:813: UserWarning: Parameter 'resnetv20_batchnorm0_beta' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/usr/local/lib/python3.6/site-packages/mxnet/gluon/parameter.py:813: UserWarning: Parameter 'resnetv20_batchnorm0_running_mean' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/usr/local/lib/python3.6/site-packages/mxnet/gluon/parameter.py:813: UserWarning: Parameter 'resnetv20_batchnorm0_running_var' is already initialized, ignoring. Set 

[18:48:13] src/operator/nn/./cudnn/./cudnn_algoreg-inl.h:97: Running performance tests to find the best convolution algorithm, this can take a while... (setting env variable MXNET_CUDNN_AUTOTUNE_DEFAULT to 0 to disable)
Epoch 0, training loss: 0.64, validation loss: 1.03, train accuracy: 0.76, validation accuracy: 0.67
['1.0296-hotdog-0000.params', '1.0296-hotdog-symbol.json']
['model.params', 'model-symbol.json']
Epoch 1, training loss: 0.53, validation loss: 0.44, train accuracy: 0.76, validation accuracy: 0.80
['1.0296-hotdog-0.states', '0.4435-hotdog-symbol.json', 'model-symbol.json', '0.4435-hotdog-0001.params', 'model.params']
['model.params', 'model-symbol.json']
Epoch 2, training loss: 0.34, validation loss: 0.40, train accuracy: 0.87, validation accuracy: 0.82
['0.4032-hotdog-symbol.json', '1.0296-hotdog-0.states', '0.4435-hotdog-symbol.json', 'model-symbol.json', '0.4032-hotdog-0002.params', '0.4435-hotdog-0001.params', '0.4435-hotdog-1.states', 'model.params']
['model.params

# Optimize the Models through Sagemaker Neo

* Sagemaker NEO compiles the models to optimize them for specific ml instance types in Sagemaker
* Here we create both a GPU optimized model and a CPU optimized model for use in conjunction with Elastic Inference

In [34]:
output_path = '/'.join(m.output_path.split('/')[:-1]) 

In [35]:
compiled_model_gpu = m.compile_model(target_instance_family='ml_p2', input_shape={'data':[1,3,512,512]}, output_path=output_path)

?.......!

In [36]:
compiled_model_cpu = m.compile_model(target_instance_family='ml_c5', input_shape={'data':[1,3,512,512]}, output_path=output_path)

?..........................!

# Model Deployment

* We deploy the models with Sagemaker's one click deployment with a few modifications to the input and output serialization 

# Model Inference Code

* Our files need to be normalized to ImageNet values for mean and standard deviations and cropped to be 224x224
* We define this code and a selection of images for use with our models
* Requires opencv package
* If this is not installed run the following code in a notebook cell

`import sys
!{sys.executable} -m pip install opencv-python`

In [47]:
filenames = glob('/home/ec2-user/SageMaker/MxNet-Sagemaker-Deployments/hotdog_not_hotdog/test/*/*')
len(filenames)

500

In [48]:
import io
import cv2 
import numpy as np

def predict_hotdog(endpoint, filenames):
    '''
    Function to preprocess and predict a list of images
    endpoint, str, Sagemaker endpoint
    filenames, list, list of images (local file locations)
    '''
    resps = []
    for img in filenames:
        img_np = cv2.imread(img)
        img_np = cv2.resize(img_np,(512,512))
        img_np = img_np.transpose(2, 0, 1)
        output_img = np.expand_dims(img_np, axis=0)
        resp = endpoint.predict(output_img)
        resps.append(resp)
    return resps

def numpy_bytes_serializer(data):
    '''
    function to serialize data for sagemaker neo endpoints
    '''
    f = io.BytesIO()
    np.save(f, data)
    f.seek(0)
    return f.read()

# Evaluating Inference on A Variety of Sagemaker Deployments

* We showcase how regular Sagemaker depoyment, EI, Neo, and EI+Neo deployments can impact endpoint latency

## Regular P2

In [140]:
predictor = m.deploy(initial_instance_count=1, 
                     instance_type='ml.p2.xlarge')

--------------------------------------------------------------------------------------------------!

In [ ]:
import time

t1 = time.time()
%timeit -n 1 predict_hotdog(predictor, filenames)
print((time.time()-t1)/60)

In [ ]:
m.delete_endpoint()

## Sagemaker NEO GPU

In [63]:
from sagemaker.predictor import npy_serializer, json_deserializer, json_serializer
compiled_predictor = compiled_model_gpu.deploy(initial_instance_count=1, 
                                               instance_type='ml.p2.xlarge')

--------------------------------------------------------------------------------------------------------------------------!

In [64]:
compiled_predictor.content_type = 'application/vnd+python.numpy+binary'
compiled_predictor.serializer = numpy_bytes_serializer

In [66]:
t1 = time.time()
%timeit -n 1 predict_hotdog(compiled_predictor, filenames)
print((time.time()-t1)/60)

56.1 s ± 18.8 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.547458950678507


In [68]:
compiled_predictor.delete_endpoint()

## Sagemaker NEO CPU

In [81]:
from sagemaker.predictor import npy_serializer, json_deserializer, json_serializer
compiled_predictor = compiled_model_cpu.deploy(initial_instance_count=1, 
                                               instance_type='ml.c5.2xlarge')

----------------------------------------------------------------------------------------------------!

In [82]:
compiled_predictor.content_type = 'application/vnd+python.numpy+binary'
compiled_predictor.serializer = numpy_bytes_serializer

In [86]:
t1 = time.time()
%timeit -n 1 predict_hotdog(compiled_predictor, filenames)
print((time.time()-t1)/60)

1min 55s ± 1.59 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
13.51935218969981


In [88]:
compiled_predictor.delete_endpoint()

## Elastic Inference

In [49]:
predictor_ei = m.deploy(initial_instance_count=1,
                        instance_type='ml.c5.2xlarge',
                        accelerator_type='ml.eia1.xlarge')

--------------------------------------------------------------------------------------------------------------!

In [95]:
t1 = time.time()
%timeit -n 1 predict_hotdog(predictor_ei, filenames)
print((time.time()-t1)/60)

3min 42s ± 20.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
25.911535314718883


In [50]:
m.delete_endpoint()

## NEO GPU + Elastic Inference

In [72]:
predictor_neo_ei = compiled_model_gpu.deploy(initial_instance_count=1, 
                                             instance_type='ml.p2.xlarge',
                                             accelerator_type='ml.eia1.xlarge')

--------------------------------------------------------------------------------------------------------------------------------------!

In [73]:
predictor_neo_ei.content_type = 'application/vnd+python.numpy+binary'
predictor_neo_ei.serializer = numpy_bytes_serializer

In [74]:
t1 = time.time()
%timeit -n 1 predict_hotdog(predictor_neo_ei, filenames)
print((time.time()-t1)/60)

56.5 s ± 19.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.591000572840373


In [79]:
predictor_neo_ei.delete_endpoint()

## NEO CPU + Elastic Inference

In [76]:
predictor_neo_ei_cpu = compiled_model_cpu.deploy(initial_instance_count=1, 
                                             instance_type='ml.c5.2xlarge',
                                             accelerator_type='ml.eia1.xlarge')

--------------------------------------------------------------------------------------------------!

In [77]:
predictor_neo_ei_cpu.content_type = 'application/vnd+python.numpy+binary'
predictor_neo_ei_cpu.serializer = numpy_bytes_serializer

In [78]:
t1 = time.time()
%timeit -n 1 predict_hotdog(predictor_neo_ei_cpu, filenames)
print((time.time()-t1)/60)

2min 4s ± 2.36 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
14.470076064268747


In [80]:
predictor_neo_ei_cpu.delete_endpoint()